# Generating text with LSTMs

In [1]:
# Run only the first time
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/albarron/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
import numpy as np
import random
import sys

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop

# In previous versions of Keras, the import was it was...
# from keras.optimizers import RMSprop

from nltk.corpus import gutenberg
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

Go to [Project Gutenberg](https://www.gutenberg.org) if you want more

In [3]:
text = ''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text += gutenberg.raw(txt).lower()
chars = sorted(list(set(text)))

# dictionary from character to index
char_indices = dict((c, i) for i, c in enumerate(chars))

# distionary from index to character
indices_char = dict((i, c) for i, c in enumerate(chars))

print('corpus length: {} total chars: {}'.format(len(text), len(chars)))

print(text[:500])

corpus length: 375542 total chars: 50
[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on


**Objective.** Predicting the character after having seen 40 characters

**Trick.** Adding redundancy to the training collection

In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
# Notice: no tokenisation; no sentence splitting; no linebreak elimination
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Total number of sequences:', len(sentences))

Total number of sequences: 125168


In [5]:
# How does an instance look like?

idx = [0, 5, 2555, 10000, 12365]

for i in idx:
    print(i)
    print(sentences[i], " --> ", next_chars[i])
    print("="*50)

0
[the tragedie of julius caesar by willia  -->  m
5
f julius caesar by william shakespeare 1  -->  5
2555
biect of my story:
i cannot tell, what y  -->  o
10000
d this, metellus
cymber

   brut. they a  -->  r
12365
hould not know you brutus. deare my lord  -->  ,


In [6]:
# Producing the one-hot encoding
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [7]:
# Building the model

model = Sequential()
model.add(LSTM(128,
    input_shape=(maxlen, len(chars))))

model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# https://keras.io/api/optimizers/rmsprop/; learning rate default: 0.001
# learning_rate used to be lr in previous versions
optimizer = RMSprop(learning_rate=0.01)

# no more binary cross entropy; no dropout (we "kind of" want to overfit here)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               91648     
_________________________________________________________________
dense (Dense)                (None, 50)                6450      
_________________________________________________________________
activation (Activation)      (None, 50)                0         
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Training 
epochs = 6
batch_size = 128
model_structure = model.to_json()

with open("shakes_lstm_model.json", "w") as json_file:
    json_file.write(model_structure)

for i in range(5):
    model.fit(X, y,
        batch_size=batch_size,
        epochs=epochs
    )
    model.save_weights("shakes_lstm_weights_{}.h5".format(i+1))
    
# It should take about 25 epochs 

Epoch 1/6
978/978 [==============================] - 62s 62ms/step - loss: 2.3554
Epoch 2/6
978/978 [==============================] - 63s 65ms/step - loss: 1.7113
Epoch 3/6
978/978 [==============================] - 65s 66ms/step - loss: 1.5819
Epoch 4/6
978/978 [==============================] - 67s 68ms/step - loss: 1.5025
Epoch 5/6
978/978 [==============================] - 66s 67ms/step - loss: 1.4644
Epoch 6/6
978/978 [==============================] - 65s 66ms/step - loss: 1.4190
Epoch 1/6
978/978 [==============================] - 67s 69ms/step - loss: 1.4180
Epoch 2/6
978/978 [==============================] - 65s 66ms/step - loss: 1.3992
Epoch 3/6
978/978 [==============================] - 64s 65ms/step - loss: 1.3802
Epoch 4/6
978/978 [==============================] - 65s 67ms/step - loss: 1.3676
Epoch 5/6
978/978 [==============================] - 62s 63ms/step - loss: 1.3567
Epoch 6/6
978/978 [==============================] - 62s 63ms/step - loss: 1.3464
Epoch 1/6
978/97

**Temperature**

temperature > 1 : more diverse outcome

temperature < 1 : more strict (try to "copy")

In [ ]:
def sample(preds, temperature=1.0):
    """Sampler to generate character sequences"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    # produces a number of random outcomes, 
    # given a probability distribution
    # n=1    number of experiments
    # preds  sequence of probabilities
    # size=1 
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)
    # Getting a random starting text
    sentence = text[start_index: start_index + maxlen]
    generated = ''
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    for i in range(400):
        # one-hot representation
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.
        
        # Producing the prediction
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        
        # looking up the next character and adding it 
        next_char = indices_char[next_index]
        generated += next_char
        
        #updating the seed
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()  # to display it right away
    print()
    
# lower values should look "more Shakesperean"

In [ ]:
# Compiling the network
model.compile('rmsprop', 
              'binary_crossentropy', 
              metrics=['accuracy'])
print(model.summary())

That's **70,200** parameters in the LSTM (against **17,550** for the RNN)

Back to the slides

In [ ]:
# Training the network

model.fit(x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test))

In [ ]:
# Saving the network for future use
model_structure = model.to_json()
with open("lstm_model1.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("lstm_weights1.h5")

In [ ]:
# Predicting
sample_1 = """I hate that the dismal weather had me down for so long, when
will it break! Ugh, when does happiness return? The sun is blinding and
the puffy clouds are too thin. I can't wait for the weekend."""

vec_list = tokenize_and_vectorize([(1, sample_1)])
test_vec_list = pad_trunc(vec_list, maxlen)
test_vec = np.reshape(
    test_vec_list, 
    (len(test_vec_list), maxlen, embedding_dims))
model.predict_classes(test_vec)


## Homework

1. Try with shorter/longer contextual sequences
2. Build a model that tries to mimic Dante